# Gradient Boosting

In [12]:
import pandas as pd

df = pd.read_csv("../data/reddit_depression_dataset_cleaned.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987011 entries, 0 to 1987010
Data columns (total 13 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   Unnamed: 0                                  int64  
 1   label                                       float64
 2   date                                        object 
 3   upvotes                                     float64
 4   num_comments                                float64
 5   combined_text                               object 
 6   tokenized_text                              object 
 7   alphanum_text                               object 
 8   stopword_removed_text                       object 
 9   stemmed_text                                object 
 10  non_stopword_removed_stemmed_text           object 
 11  combined_stemmed_text                       object 
 12  combined_non_stopword_removed_stemmed_text  object 
dtypes: float64(3), int64(1), ob

In [13]:
df.head()

,Unnamed: 0,label,date,upvotes,num_comments,combined_text,tokenized_text,alphanum_text,stopword_removed_text,stemmed_text,non_stopword_removed_stemmed_text,combined_stemmed_text,combined_non_stopword_removed_stemmed_text
0,0,0.0,2014-07-14 03:35:09,4.0,0.0,Deep thoughts underdog Only when we start cons...,"['deep', 'thoughts', 'underdog', 'only', 'when...","['deep', 'thoughts', 'underdog', 'only', 'when...","['deep', 'thoughts', 'underdog', 'start', '99'...","['deep', 'thought', 'underdog', 'start', '99',...","['deep', 'thought', 'underdog', 'onli', 'when'...",deep thought underdog start 99 underdog start ...,deep thought underdog onli when we start consi...
1,1,0.0,2014-09-13 00:31:19,4.0,1.0,"I like this sub, there's only two posts yet I ...","['i', 'like', 'this', 'sub', ',', ""there's"", '...","['i', 'like', 'this', 'sub', 'only', 'two', 'p...","['posts', 'coming', 'human', 'morality', 'joke...","['post', 'come', 'human', 'moral', 'joke', 'lo...","['i', 'like', 'thi', 'sub', 'onli', 'two', 'po...",post come human moral joke long abscenc hope d...,i like thi sub onli two post yet i keep come b...
2,2,0.0,2014-11-20 04:31:58,6.0,1.0,Rebirth! Hello. \nI am the new guy in charge h...,"['rebirth', '!', 'hello', '.', 'i', 'am', 'the...","['rebirth', 'hello', 'i', 'am', 'the', 'new', ...","['rebirth', 'guy', 'charge', 'thegood', 'ofc',...","['rebirth', 'guy', 'charg', 'thegood', 'ofc', ...","['rebirth', 'hello', 'i', 'am', 'the', 'new', ...",rebirth guy charg thegood ofc bring weirdpinea...,rebirth hello i am the new guy in charg here b...
3,3,0.0,2014-11-20 19:38:05,25.0,2.0,"""I want to be like water. I want to slip throu...","['""', 'i', 'want', 'to', 'be', 'like', 'water'...","['i', 'want', 'to', 'be', 'like', 'water', 'i'...","['water', 'slip', 'fingers', 'hold', 'ship', '...","['water', 'slip', 'finger', 'hold', 'ship', 'm...","['i', 'want', 'to', 'be', 'like', 'water', 'i'...",water slip finger hold ship michel william,i want to be like water i want to slip through...
4,5,0.0,2014-11-22 19:17:39,8.0,23.0,What is the limit of the knowledge and power a...,"['what', 'is', 'the', 'limit', 'of', 'the', 'k...","['what', 'is', 'the', 'limit', 'of', 'the', 'k...","['limit', 'knowledge', 'power', 'human', 'pers...","['limit', 'knowledg', 'power', 'human', 'perso...","['what', 'is', 'the', 'limit', 'of', 'the', 'k...",limit knowledg power human person infinit grow,what is the limit of the knowledg and power a ...


In [18]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [15]:
y = df['label']

In [16]:
# Fill NaN values in the text column with an empty string
df['combined_non_stopword_removed_stemmed_text'].fillna('', inplace=True)


0          deep thought underdog onli when we start consi...
1          i like thi sub onli two post yet i keep come b...
2          rebirth hello i am the new guy in charg here b...
3          i want to be like water i want to slip through...
4          what is the limit of the knowledg and power a ...
                                 ...                        
1987006    intent car crash yesterday i just got in my ca...
1987007    alon as usual i just argu with my mom an hour ...
1987008    help need urgent ok basic non of my or famili ...
1987009    i know what your think do it not today it gonn...
1987010          2022 year in review it suck god help us all
Name: combined_non_stopword_removed_stemmed_text, Length: 1987011, dtype: object

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(df['combined_non_stopword_removed_stemmed_text'])


ValueError: np.nan is an invalid document, expected byte or unicode string.

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 6,
    'eta': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42
}


In [8]:
evals = [(dtrain, 'train'), (dtest, 'eval')]

bst = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=evals,
    early_stopping_rounds=10,
    verbose_eval=10
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred_prob = bst.predict(dtest)
y_pred = (y_pred_prob > 0.5).astype(int)


In [ ]:
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)


In [9]:
from sklearn.metrics import accuracy_score, classification_report

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Detailed classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.9460748912313194
              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97    352228
         1.0       0.84      0.65      0.73     45175

    accuracy                           0.95    397403
   macro avg       0.90      0.82      0.85    397403
weighted avg       0.94      0.95      0.94    397403



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(X_train, y_train)

print(f"Best params: {grid_search.best_params_}")
